# Extraction of data from RDS-file

Run this notebook in a R kernel.
In the case the data is provides as a RDS-file use the following procedure to extract the components of the sparse matrix, i.e. the counts and the position of the counts

## First load the object and the relevant subcomponents

In [ ]:
library(SummarizedExperiment)
library(GenomicRanges)
library(stats4)
library(BiocGenerics)
library(parallel)

In [3]:
Exp <- readRDS('/vol/storage/projects/Including_ATAC/Trevino/Datasets/ATAC.RDS')

In [4]:
Exp

Loading required package: SingleCellExperiment


class: SingleCellExperiment 
dim: 657930 31304 
metadata(0):
assays(1): ''
rownames(657930): c2_peak_1 c4_peak_1 ... c12_peak_153318
  c2_peak_115295
rowData names(11): name score ... score_norm classification
colnames(31304): hft_w20_p3_r1_cell_0 hft_w20_p3_r1_cell_1 ...
  hft_w16_p7_r3_cell_2864 hft_w16_p7_r3_cell_2865
colData names(28): Cell.ID Sample.ID ... Cell.Barcode
  TSS.Enrichment.Unsmoothed
reducedDimNames(2): PCA UMAP
spikeNames(0):
altExpNames(0):

In [11]:
colData(Exp)

DataFrame with 31304 rows and 28 columns
                                        Cell.ID     Sample.ID         Age
                                    <character>   <character> <character>
hft_w20_p3_r1_cell_0       hft_w20_p3_r1_cell_0 hft_w20_p3_r1       pcw20
hft_w20_p3_r1_cell_1       hft_w20_p3_r1_cell_1 hft_w20_p3_r1       pcw20
hft_w20_p3_r1_cell_2       hft_w20_p3_r1_cell_2 hft_w20_p3_r1       pcw20
hft_w20_p3_r1_cell_3       hft_w20_p3_r1_cell_3 hft_w20_p3_r1       pcw20
hft_w20_p3_r1_cell_4       hft_w20_p3_r1_cell_4 hft_w20_p3_r1       pcw20
...                                         ...           ...         ...
hft_w16_p7_r3_cell_2860 hft_w16_p7_r3_cell_2860 hft_w16_p7_r3       pcw16
hft_w16_p7_r3_cell_2862 hft_w16_p7_r3_cell_2862 hft_w16_p7_r3       pcw16
hft_w16_p7_r3_cell_2863 hft_w16_p7_r3_cell_2863 hft_w16_p7_r3       pcw16
hft_w16_p7_r3_cell_2864 hft_w16_p7_r3_cell_2864 hft_w16_p7_r3       pcw16
hft_w16_p7_r3_cell_2865 hft_w16_p7_r3_cell_2865 hft_w16_p7_r3       pcw

In [12]:
write.csv(x=rowData(Exp), file="/vol/storage/projects/Including_ATAC/Trevino/Datasets/rowData")

In [13]:
write.csv(x=colData(Exp), file="/vol/storage/projects/Including_ATAC/Trevino/Datasets/colData")

In [30]:
lengths(reducedDims(Exp))

PCA  UMAP 
31304 31304

In [35]:
write.csv(x=reducedDims(Exp)[2], file="/vol/storage/projects/Including_ATAC/Trevino/Datasets/reduced_UMAP")

In [6]:
rownames(Exp) <- gsub(" ", "",paste(as.vector(seqnames(rowRanges(Exp))),':',start(ranges(rowRanges(Exp))),'-',end(ranges(rowRanges(Exp)))), fixed=TRUE)

In [7]:
write.csv(x=rownames(Exp), file="/vol/storage/projects/Including_ATAC/Trevino/Datasets/Chromos")

In [9]:
write.csv(x=colnames(Exp), file="/vol/storage/projects/Including_ATAC/Trevino/Datasets/Cells")

## Next identify the relevant components needed to re-create the sparse count matrix

The components are the counts.

counts@x are the matrix-entries. The n-th number in the list represents the experimental counts of the n-th 
    entry of the count matrix, where the ennumaration has been done coloumn-wise.

counts@i are the coloumn-positions. The n-th number in the list represents the column of the n-th 
    entry of the count matrix.

counts@p is the amount of entries per column. counts@p[0]=0, and the n-th entry in counts@p[2:] represents the 
    total amount of entries in the n-th column of the count matrix

In [6]:
counts <- assay(Exp)

In [14]:
counts

   [[ suppressing 31 column names ‘hft_w20_p3_r1_cell_0’, ‘hft_w20_p3_r1_cell_1’, ‘hft_w20_p3_r1_cell_2’ ... ]]
   [[ suppressing 31 column names ‘hft_w20_p3_r1_cell_0’, ‘hft_w20_p3_r1_cell_1’, ‘hft_w20_p3_r1_cell_2’ ... ]]
   [[ suppressing 31 column names ‘hft_w20_p3_r1_cell_0’, ‘hft_w20_p3_r1_cell_1’, ‘hft_w20_p3_r1_cell_2’ ... ]]


657930 x 31304 sparse Matrix of class "dgCMatrix"
                                                                                           
chr1:10028-10528     . . . . . . . . .  . . . . .  . . . . . . . . . . . . . . . . . ......
chr1:180647-181147   . . . . . . . . .  . . . . .  . . . . . . . . . . . . . . . . . ......
chr1:181234-181734   . . . . . . . . .  . . . . .  . . . . . . . . . . . . . . . . . ......
chr1:191218-191718   . . . . . . . . .  . . . . .  . . . . . . . . . . . . . . . . . ......
chr1:265926-266426   . . . . . . . . .  . . . . .  . . . . . . . . . . . . . . . . . ......
chr1:267751-268251   . . . . . . . . .  . . . . .  . . . . . . . . . . . . . . . . . ......
chr1:585947-586447   . . . . . . . . .  . . . . .  . . . . . . . . . . . . . . . . . ......
chr1:629697-630197   . . . . . . . . .  . . . . .  . . . . . . . . . . . . . . . . . ......
chr1:633777-634277   . . . . . . . . .  . . . . .  . . . . . . . . 2 . . . . . . . . ......
chr1:778422-778922   2 2 . 2 3

In [23]:
a1=counts@x[1:367728796]

In [50]:
counts@p[367728795]

[1] 657856

In [32]:
counts@x[0]

numeric(0)

In [24]:
write.csv(x=a1, file="/vol/storage/projects/Including_ATAC/Trevino/Datasets/Counts_N")

In [38]:
a2=counts@i[1:367728796]

In [41]:
write.csv(x=a2, file="/vol/storage/projects/Including_ATAC/Trevino/Datasets/Counts_R")

In [47]:
counts@p[2]

[1] 25600

In [ ]:
a3=counts@i[2:31304]

In [36]:
write.csv(x=a3, file="/vol/storage/projects/Including_ATAC/Trevino/Datasets/Counts_C")